# Computer Vision Recap - Segmentation

## Lecture 13 Isotropic Nonlinear Diffusion

### Isotropic Nonlinear Diffusion


Avoid drawbacks of linear diffusion: delocalisation and blurring of edges
$$ \partial_t u = div( g(|\nabla u|^2) \; \nabla u ) $$

Diffusivity $g(|\nabla u|^2) := \frac{1}{1+|\nabla u|^2/\lambda^2}$ decreasing in $|\nabla u|^2$ 


<img src="./images/isodiff.png" width=600>
<br>

- Forward diffusion if $\Phi'(u_x) > 0$ for $|\nabla u| < \lambda$
- Backward diffusion if $\Phi'(u_x) < 0$ for $|\nabla u| > \lambda$

### Regularised Nonlinear Diffusion

- Reduce staircasing artifacts and problems with noise

$$ \partial_t u = div( g(|\nabla u_\sigma|^2) \; \nabla u ) $$ with $$ u_\sigma = K_\sigma*u $$

<br>

Theoretical Results

- Stability under perturbations of initial image
- Preservation of average grey value
- Maximum-minimum principle
- Stable
- Image converges to average grey level

<br><br>
## Anisotropic Nonlinear Diffusion

**Motivation**
- Isotropic nonlinear diff. have difficulties with noisy images and cannot enhance coherent flow-like structures
- We want to smooth only along edge directions but no across. 

<br>

### Edfe-enhancing Anisotropic Diffusion (EED)

**Goal**
- Smooth prefently within regions
- Inhibit diffusion perpendicular to edges

$$ D(\nabla u_\sigma) = (v_1 \;\; v_2) \begin{bmatrix} \lambda_1&0\\0&\lambda_2 \end{bmatrix} (v_1 \;\; v_2)^\top = \lambda_1 v_1 v_1^\top + \lambda_2 v_2 v_2^\top $$

- $\lambda_1 = 1$ (diffusion along edge), $v_1 \parallel \nabla u_\sigma$ 
- $\lambda_2 = g(|\nabla u|^2)$ (diffusion across edge), $v_2 \perp \nabla u_\sigma$








